<a href="https://colab.research.google.com/github/Vampaxx/Pyspark_basics/blob/main/MapType_Column.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3271b3430fb08a7f628a55d2a7f74468aa5ee894210655cd6ae50d2fe19eb4bb
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [14]:
from pyspark.sql.types import (StringType,StructField,
                               StructType,IntegerType,
                               ArrayType,MapType)
from pyspark.sql import SparkSession

from pyspark.sql.functions import col,explode,split
from pyspark.sql.functions import col,lit,when,expr



In [4]:
spark = SparkSession.builder.appName('MapType_Column').getOrCreate()

## MapType Column

Pyspark Maptype is used to represent map key-value pair similar to python Dictionary(Dict)

In [5]:
from os import truncate
data = [('Arjun',{'hair': 'black', 'eye': 'brown'}),\
        ('Micheal',{'hair': 'black', 'eye':'blue'})]

schema = ['Emp_name','properties']
df = spark.createDataFrame(data,schema)
df.show(truncate=False)
df.printSchema()

+--------+-----------------------------+
|Emp_name|properties                   |
+--------+-----------------------------+
|Arjun   |{eye -> brown, hair -> black}|
|Micheal |{eye -> blue, hair -> black} |
+--------+-----------------------------+

root
 |-- Emp_name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [6]:
schema = StructType([\
                     StructField('Emp_name',StringType()),\
                     StructField('properties',MapType(StringType(),StringType() )) ])

df = spark.createDataFrame(data,schema)
df.printSchema()
df.show(truncate=False)

root
 |-- Emp_name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+--------+-----------------------------+
|Emp_name|properties                   |
+--------+-----------------------------+
|Arjun   |{eye -> brown, hair -> black}|
|Micheal |{eye -> blue, hair -> black} |
+--------+-----------------------------+



In [7]:
df.withColumn('eye',df.properties['eye']).show(truncate=False)

+--------+-----------------------------+-----+
|Emp_name|properties                   |eye  |
+--------+-----------------------------+-----+
|Arjun   |{eye -> brown, hair -> black}|brown|
|Micheal |{eye -> blue, hair -> black} |blue |
+--------+-----------------------------+-----+



## Explode the mapped column

In [20]:
df.select('Emp_name',explode('properties').alias('Organ','properties')).show()

+--------+-----+----------+
|Emp_name|Organ|properties|
+--------+-----+----------+
|   Arjun|  eye|     brown|
|   Arjun| hair|     black|
| Micheal|  eye|      blue|
| Micheal| hair|     black|
+--------+-----+----------+

